In [1]:
import os
os.chdir("../")

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mohamedmagri/orangePlatform.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mohamedmagri"
os.environ["MLFLOW_TRACKING_PASSWORD"]="c0883b6d816c6c5ba7db4860a97f4d36c5fe5904"

In [37]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    mlflow_uri: str
    num_lstm_units: float
    learning_rate: float
    epochs: float
    early_stopping_patience: float
    nsteps: float


In [38]:
from orangePlatform.constants import *
from orangePlatform.utils.common import read_yaml, create_directories, save_json

In [39]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ConvLSTM2D

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            mlflow_uri="https://dagshub.com/mohamedmagri/orangePlatform.mlflow",
            num_lstm_units=params.num_lstm_units,
            learning_rate=params.learning_rate,
            epochs=params.epochs,
            early_stopping_patience=params.early_stopping_patience,
            nsteps=params.nsteps
           
        )

        return model_evaluation_config


In [40]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [41]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [42]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test=test_data.values
        test_x,test_y=split_sequence(test, self.config.nsteps)
        # test_x = test_data.drop([self.config.target_column], axis=1)
        # test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            n_seq =5
            n_features=1
            n_steps = int(self.config.nsteps // n_seq)
            x_input = test_x.reshape((test_x.shape[0], n_seq, 1, n_steps, n_features))


            yhat = model.predict(x_input, verbose=0)

            (rmse, mae, r2) = self.eval_metrics(test_y, yhat)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ConvLSTM2D")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [43]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-05-11 02:20:03,384: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 02:20:03,390: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 02:20:03,393: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-11 02:20:03,395: INFO: common: created directory at: artifacts]
[2024-05-11 02:20:03,396: INFO: common: created directory at: artifacts/model_evaluation]
[2024-05-11 02:20:04,163: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ConvLSTM2D' already exists. Creating a new version of this model...
2024/05/11 02:20:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ConvLSTM2D, version 2
Created version '2' of model 'ConvLSTM2D'.
